## Assignment 3

1.复习课上内容及代码。

2.回答以下理论题目：

### 2.1 动态规划(DP)可以处理的问题有什么特点 ？
(Divide & Conquer + Optimal Substructure)
1. 把相同重复的子问题构成的大问题划分成多个阶段处理 
2. 求得的阶段最优的解也是全局最优


### 2.2 简单描述动态规划的几个步骤？
1. 找到重复子问题
2. 定义状态转移方程

2.3 编辑距离算法的迭代公式是什么？

if w1\[i\]==w2\[j\]
    edit_dist(i, j) = edit_dist(i-1, j-1)
else
    edit_dist(i, j) =
        edit_dist(i-1, j-1) + 1
        edit_dist(i-1, j) + 1
        edit_dist(i, j-1) + 1

3.实践题

3.1 完成编辑距离解码函数

In [1]:
from functools import lru_cache

## 老师的代码

In [2]:
solution = {}
def edit_distance(string1, string2):
    solution = {}
    @lru_cache(maxsize=2**10)
    def edit_distance_(string1, string2):

        if len(string1) == 0: 
            cost = len(string2)
            while len(string2) > 0:
                solution[("", string2)] = "ADD {}".format(string2[-1])
                string2 = string2[:-1]
            return cost
        if len(string2) == 0: 
            cost = len(string1)
            while len(string1) > 0:
                solution[("", string1)] = "DEL {}".format(string1[-1])
                string1 = string1[:-1]
            return cost

        tail_s1 = string1[-1]
        tail_s2 = string2[-1]

        candidates = [
            (edit_distance_(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
            # string 1 delete tail
            (edit_distance_(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
            # string 1 add tail of string2
        ]
        if tail_s1 == tail_s2:
            both_forward = (edit_distance_(string1[:-1], string2[:-1]) + 0, '')
        else:
            both_forward = (edit_distance_(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

        candidates.append(both_forward)

        min_distance, operation = min(candidates, key=lambda x: x[0])

        solution[(string1, string2)] = operation 

        return min_distance
    min_distance = edit_distance_(string1, string2)
    return solution, min_distance


In [3]:
s1 = 'ABCDECG'
s2 = 'ABCCEF'
edit_distance(s1, s2)[0]

{('', 'ABCCEF'): 'ADD F',
 ('', 'ABCCE'): 'ADD E',
 ('', 'ABCC'): 'ADD C',
 ('', 'ABC'): 'DEL C',
 ('', 'AB'): 'DEL B',
 ('', 'A'): 'DEL A',
 ('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('', 'ABCD'): 'DEL D',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('', 'ABCDE'): 'DEL E',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('', 'ABCDEC'): 'DEL C',
 ('A

In [4]:
def parse_edit_solution(s1, s2, solution):   
    def parse_edit_solution_(s1, s2):
        if s1 == s2:
            return []
        action = solution[(s1, s2)]
        if "DEL" in action:
            return [action] + parse_edit_solution_(s1[:-1], s2)
        elif "ADD" in action:
            return [action] + parse_edit_solution_(s1, s2[:-1])
        elif "SUB" in action:
            return [action] + parse_edit_solution_(s1[:-1], s2[:-1])
        else:
            return parse_edit_solution_(s1[:-1], s2[:-1])


    actions = parse_edit_solution_(s1, s2)

    return actions
            

In [5]:
def edit_distance_and_parse(string1, string2):
    solution, min_dis = edit_distance(string1, string2)
    actions = parse_edit_solution(string1, string2, solution)
    return actions, min_dis

In [6]:
edit_distance_and_parse(s1, s2)

(['DEL G', 'SUB C => F', 'SUB D => C'], 3)

3.2 完成拼音自动纠错代码

如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

In [17]:
import pinyin
import re
from collections import Counter, defaultdict

In [13]:
chinese_dataset = 'article_9k.txt'
CHINESE_CHARATERS = open(chinese_dataset, encoding="utf8").read()

In [14]:
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [16]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [18]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [ ]:
NEW_PINYIN_COUNT = {}
for k in PINYIN_COUNT.keys():
    if len(k) != 1:
        NEW_PINYIN_COUNT[k] = PINYIN_COUNT[k]
NEW_PINYIN_COUNT = Counter(NEW_PINYIN_COUNT)

In [19]:
tokens(CHINESE_CHARATERS_COPYS[:100])

['ci',
 'wai',
 'zi',
 'ben',
 'zhou',
 'yue',
 'ri',
 'qi',
 'chu',
 'xiao',
 'mi',
 'shou',
 'ji',
 'deng',
 'kuan',
 'ji',
 'xing',
 'wai',
 'qi',
 'yu',
 'ji',
 'xing',
 'yi']

In [24]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in NEW_PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)   
    deletes = [a+b[1:] for (a,b) in pairs if b] # input pinyin -删除操作 去掉b[0]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b] #替换  b[0]替换为c
    inserts = [a+c+b for (a,b) in pairs for c in alphabet] #插入操作 在a和b之间插入字母
    return set(deletes + replaces + inserts)

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)} #要计算编辑距离为2的词 可以通过计算与原词编辑距离为1的词的编辑距离为1的词

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]


In [25]:
def correct(word): #pinyin candidates piyin pingyin aogyin .... 
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])  #计算所有与输入词编辑距离为0，1，2的词
    return max(candidates,key=NEW_PINYIN_COUNT.get) #取出在所有candidate里在语料库里出现次数最多的词

In [23]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [26]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [27]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe si yi ge ce shi'

In [28]:
TOKEN = tokens(CHINESE_CHARATERS_COPYS)

In [31]:
words_count = Counter(TOKEN)
new_words_count = {}
for x in words_count.keys():
    if len(x) > 1:
        new_words_count[x] = words_count[x]

In [30]:
TOKEN_2_GRAM = ['/'.join(TOKEN[i:i+2]) if (len(TOKEN[i]) > 1 and len (TOKEN[i+1]) > 1) else "" for i in range(len(TOKEN[:-2]))]

In [51]:
TOKEN_2_GRAM

['ciwai',
 'waizi',
 'ziben',
 'benzhou',
 'zhouyue',
 'yueri',
 'riqi',
 'qichu',
 'chuxiao',
 'xiaomi',
 'mishou',
 'shouji',
 'jideng',
 'dengkuan',
 'kuanji',
 'jixing',
 'xingwai',
 'waiqi',
 'qiyu',
 'yuji',
 'jixing',
 'xingyi',
 'yizan',
 'zanting',
 'tinggeng',
 'gengxin',
 'xinfa',
 'fabu',
 'buhan',
 'hankai',
 'kaifa',
 'faban',
 'banti',
 'tiyan',
 'yanban',
 'bannei',
 'neice',
 'cewen',
 'wending',
 'dingban',
 'banzan',
 'zanbu',
 'bushou',
 'shouying',
 'yingxiang',
 'xiangyi',
 'yique',
 'quebao',
 'baogong',
 'gongcheng',
 'chengshi',
 'shike',
 'keyi',
 'yiji',
 'jizhong',
 'zhongquan',
 'quanbu',
 'bujing',
 'jingli',
 'lijin',
 'jinxing',
 'xingxi',
 'xitong',
 'tongyou',
 'youhua',
 'huagong',
 'gongzuo',
 'zuoyou',
 'youren',
 'rencai',
 'caice',
 'cezhe',
 'zheye',
 'yeshi',
 'shijiang',
 'jiangjing',
 'jingli',
 'lizhu',
 'zhuyao',
 'yaoyong',
 'yongdao',
 '',
 '',
 '',
 '',
 '',
 'deyan',
 'yanfa',
 'fazhi',
 'zhizhong',
 '',
 '',
 '',
 '',
 '',
 'qunian',
 '

In [32]:
word_counter_2 = Counter(TOKEN_2_GRAM)

In [50]:
word_counter_2

Counter({'ciwai': 4289,
         'waizi': 913,
         'ziben': 3401,
         'benzhou': 689,
         'zhouyue': 1461,
         'yueri': 165274,
         'riqi': 3199,
         'qichu': 825,
         'chuxiao': 294,
         'xiaomi': 376,
         'mishou': 68,
         'shouji': 6003,
         'jideng': 1415,
         'dengkuan': 8,
         'kuanji': 226,
         'jixing': 3120,
         'xingwai': 138,
         'waiqi': 475,
         'qiyu': 1583,
         'yuji': 8095,
         'xingyi': 3683,
         'yizan': 101,
         'zanting': 874,
         'tinggeng': 31,
         'gengxin': 906,
         'xinfa': 1688,
         'fabu': 13302,
         'buhan': 304,
         'hankai': 72,
         'kaifa': 6761,
         'faban': 192,
         'banti': 114,
         'tiyan': 4191,
         'yanban': 120,
         'bannei': 30,
         'neice': 30,
         'cewen': 144,
         'wending': 4619,
         'dingban': 127,
         'banzan': 5,
         'zanbu': 198,
         'bushou':

In [48]:
def prob_1(word):
    if word in new_words_count:
        return new_words_count[word] / len(TOKEN)
    else:
        return 1/len(TOKEN)/10000
        
def prob_2(word1, word2):
    probs1 = prob_1(word2)
    word2gram = '/'.join([word1, word2])
    print(word2gram)
    if word2gram in word_counter_2:
        return word_counter_2[word2gram] / len(TOKEN_2_GRAM) * probs1
    else:
        return 1 / len(TOKEN_2_GRAM) * probs1

In [52]:
prob_1("haoxiang")

3.1975670863164366e-12

In [54]:
prob_2("hao", "xiang")

hao/xiang


2.0112590153136057e-10

In [47]:
prob_2("zhou", "shang")

1.8295707889199483e-10

In [69]:
def split_sequence(string):
    solution = {}
    @lru_cache(maxsize = 2**10)
    def split_sequence_(string):
        print(string)
        max_count, max_split = max([(prob_1(string), 0)] + \
                                   [(prob_2(split_sequence_(string[:i])[1], split_sequence_(string[i:])[1]), i) for i in range(1, len(string))], 
                                key = lambda x: x[0])  
        solution[string] = (string[:max_split], string[max_split:])
        return max_count, string

    max_count = split_sequence_(string)
    return solution, max_count
                                                            

In [36]:
def parse_split_result(string, solution):
    def parse_split_result_(string):
        left_string, right_string = solution[string]
        if len(left_string) == 0:
            return solution[right_string]
        return parse_split_result_(left_string) + parse_split_result_(right_string)
    return parse_split_result_(string)

In [37]:
def split_and_parse_sequence(string):
    solution, _ = split_sequence(string)
    string_list = parse_split_result(string, solution)
    return string_list

In [70]:
split_and_parse_sequence("weyaoqu")

weyaoqu
w
eyaoqu
e
yaoqu
y
aoqu
a
oqu
o
qu
q
u
q/u
o/qu
oq
o/q
oq/u
a/oqu
ao
a/o
ao/qu
aoq
a/oq
ao/q
aoq/u
y/aoqu
ya
y/a
ya/oqu
yao
y/ao
ya/o
yao/qu
yaoq
y/aoq
ya/oq
yao/q
yaoq/u
e/yaoqu
ey
e/y
ey/aoqu
eya
e/ya
ey/a
eya/oqu
eyao
e/yao
ey/ao
eya/o
eyao/qu
eyaoq
e/yaoq
ey/aoq
eya/oq
eyao/q
eyaoq/u
w/eyaoqu
we
w/e
we/yaoqu
wey
w/ey
we/y
wey/aoqu
weya
w/eya
we/ya
wey/a
weya/oqu
weyao
w/eyao
we/yao
wey/ao
weya/o
weyao/qu
weyaoq
w/eyaoq
we/yaoq
wey/aoq
weya/oq
weyao/q
weyaoq/u


('', 'we', '', 'yao', '', 'qu')